In [1]:
import os
import random
from pathlib import Path
from argparse import ArgumentParser
import pandas as pd
from urllib.request import urlopen
from PIL import Image
import matplotlib.pyplot as plt
import timm.utils
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Function to set the seed for reproducibility
def set_seed(seed_value=42):
    """Set seed for reproducibility."""
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)  # if you are using multi-GPU.
    random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    # The below two lines are for deterministic algorithm behavior in CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
data_dir = "data/training/dataset"
nbr_classes = len(os.listdir(data_dir))
nbr_classes

7806

In [4]:
def load_class_mapping(class_list_file):
    with open(class_list_file) as f:
        class_index_to_class_name = {i: line.strip() for i, line in enumerate(f)}
    return class_index_to_class_name


def load_species_mapping(species_map_file):
    df = pd.read_csv(species_map_file, sep=';', quoting=1, dtype={'species_id': str})
    df = df.set_index('species_id')
    return  df['species'].to_dict()

cid_to_spid = load_class_mapping("models/pretrained_models/class_mapping.txt")
spid_to_sp = load_species_mapping("models/pretrained_models/species_id_to_name.txt")

In [5]:
data_transform = transforms.Compose([
    # Resize the images to 518x518
    transforms.Resize(size=(518,518)),
    # Flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5),  # p = probability of flip, 0.5 = 50% chance
    # Add your custom augmentation here
    transforms.RandomApply([transforms.TrivialAugmentWide(num_magnitude_bins=31)], p=0.5),
    # Turn the image into a torch.Tensor
    transforms.ToTensor(),
    # Normalize the image
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def one_hot(label_idx, nbr_classes):
    one_hot_tensor = torch.zeros(nbr_classes)
    one_hot_tensor.scatter_(0, torch.tensor(label_idx), value=1)
    return one_hot_tensor

label_transformation = transforms.Compose([transforms.Lambda(lambda y: one_hot(y, nbr_classes))])

dataset = datasets.ImageFolder(data_dir,
                            transform=data_transform,
                            target_transform= label_transformation)

In [6]:
img, label = dataset[0][0], dataset[0][1]
print(f"Image tensor:\n{img}")
print(f"Image shape: {img.shape}")
print(f"Image datatype: {img.dtype}")
print(f"Image label: {label}")
print(f"Label datatype: {type(label)}")

Image tensor:
tensor([[[-0.5596, -0.8164, -0.4397,  ..., -2.1179, -2.1179, -2.1179],
         [-0.5424, -0.4568, -0.3712,  ..., -2.1179, -2.1179, -2.1179],
         [-0.5938, -0.4911, -0.5596,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-1.2788, -1.3815, -1.5185,  ..., -2.1179, -2.1179, -2.1179],
         [-1.3644, -1.3473, -1.4329,  ..., -2.1179, -2.1179, -2.1179],
         [-1.4672, -1.4158, -1.4329,  ..., -2.1179, -2.1179, -2.1179]],

        [[ 0.5028,  0.1352,  0.4853,  ..., -2.0357, -2.0357, -2.0357],
         [ 0.6254,  0.6429,  0.7129,  ..., -2.0357, -2.0357, -2.0357],
         [ 0.7129,  0.7654,  0.6779,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-0.3725, -0.5476, -0.6877,  ..., -2.0357, -2.0357, -2.0357],
         [-0.6352, -0.6527, -0.7402,  ..., -2.0357, -2.0357, -2.0357],
         [-0.8627, -0.7752, -0.8102,  ..., -2.0357, -2.0357, -2.0357]],

        [[-0.4275, -0.6715, -0.2881,  ..., -1.8044, -1.8044, -1.8044],
         [-0.3578, -0.2707, -0.

In [7]:
def create_writer( experiment_name: str, model_name: str, extra: str=None):
    timestamp = datetime.now().strftime("%Y-%m-%d")
    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)
    
    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

In [8]:
writer = create_writer("Run_3", "vit_small_patch14_reg4_dinov2", "lr-8.0e-05_epoch-100_batch-12")

[INFO] Created SummaryWriter, saving to: runs\2024-05-26\Run_3\vit_small_patch14_reg4_dinov2\lr-8.0e-05_epoch-100_batch-12...


In [9]:
class CreatDataloader(Dataset):
    def __init__(self,
                 dataset,
                 batch:int
    ) -> None:
        TRAIN_PERCENT = 00.4
        self.dataset = dataset
        self.batch = batch
        self.dataset_size = len(dataset)
        self.train_size = int(TRAIN_PERCENT * self.dataset_size)
        self.test_size = int(0.01 * self.dataset_size)
        # self.test_size = int((self.dataset_size - self.train_size)/2)
        self.validation_size = self.dataset_size - self.train_size - self.test_size
        self.train_data, self.test_data, self.validation_data = torch.utils.data.random_split(self.dataset,
                                                                                              [self.train_size, self.test_size, self.validation_size])
        
    def get_train_dataloader(self):
        return DataLoader(self.train_data,
                          batch_size=self.batch,
                          shuffle=True,
                          pin_memory=True)
    
    def get_test_dataloader(self):
        return DataLoader(self.test_data,
                          batch_size=self.batch,
                          shuffle=False,
                          pin_memory=True)
        
    def get_validation_dataloader(self):
        return DataLoader(self.validation_data,
                          batch_size=self.batch,
                          shuffle=False,
                          pin_memory=True)
        

In [10]:
set_seed()
dataloader = CreatDataloader(dataset, batch=12)
training_dataloader = dataloader.get_train_dataloader()
test_dataloader = dataloader.get_test_dataloader()
validation_dataloader = dataloader.get_validation_dataloader()
loss_fn = torch.nn.BCELoss()

model = timm.create_model('vit_small_patch14_reg4_dinov2.lvd142m',
                          pretrained=True,
                          num_classes=0)

model.head = nn.Sequential(nn.Linear(model.num_features, nbr_classes), nn.Sigmoid())

optimizer = torch.optim.Adam(model.parameters(), lr=8.0e-05)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=6)

In [11]:
class Trainer:
    def __init__(
        self,
        model: torch.nn.Module,
        train_data: DataLoader,
        test_data: DataLoader,
        optimizer: torch.optim.Optimizer,
        loss_fn: torch.nn.Module,
        scheduler: torch.optim.lr_scheduler,
        writer: SummaryWriter
    ) -> None:
        self.model = model.to(device)
        self.train_data = train_data
        self.test_data = test_data
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.epochs_run = 0
        self.scheduler = scheduler
        self.writer = writer
        self.step = 0

    def _run_batch(self, source, targets, idx):
        NUM_ACCUMULATION_STEPS = 10
        self.optimizer.zero_grad()
        output = self.model(source)
        loss = self.loss_fn(output, targets)
        loss = loss / NUM_ACCUMULATION_STEPS
        train_pred_labels = torch.round(output)
        loss.backward()
        if ((idx+1) % NUM_ACCUMULATION_STEPS == 0) or (idx+1 == len(self.train_data)):
            self.optimizer.step()
        
        if self.writer:
            self.writer.add_scalar("Batch/Loss", loss.item(), self.step)
        self.step = self.step+1
        return loss.item(), (torch.all(train_pred_labels == targets, dim=1).sum().item()/len(train_pred_labels))
    # print(torch.all(y==sample[1].to(device), dim=1).sum().item()/len(y))
        
        
    def _test_batch(self, source, targets):
        test_output = self.model(source)
        loss = self.loss_fn(test_output, targets)
        test_pred_labels = torch.round(test_output)
        return loss.item(), (torch.all(test_pred_labels == targets, dim=1).sum().item()/len(test_pred_labels))
        

    def _run_epoch(self, epoch):
        test_loss, test_acc = 0, 0
        train_loss, train_acc = 0, 0
        b_sz = len(next(iter(self.train_data))[0])
        print(f"[GPU{device}] Epoch {epoch} | Batchsize: {b_sz} | Steps: {len(self.train_data)}")
        for idx, sample in enumerate(self.train_data):
            source = sample[0].to(device)
            targets = sample[1].to(device)
            loss, accu = self._run_batch(source, targets, idx)
            train_loss += loss
            train_acc += accu
        self.scheduler.step()
        train_loss = train_loss / len(self.train_data)
        train_acc = train_acc / len(self.train_data)
            
        if epoch % 3 == 0:
            self.model.eval()
            with torch.inference_mode():
                for source, targets in self.test_data:
                    source = source.to(device)
                    targets = targets.to(device)
                    loss, accu = self._test_batch(source, targets)
                    test_loss += loss
                    test_acc += accu
            test_loss = test_loss / len(self.test_data)
            test_acc = test_acc / len(self.test_data)
            if self.writer:
                self.writer.add_scalar("Loss/test", test_loss, epoch)
                self.writer.add_scalar("Accuracy/test", test_acc, epoch)
            
            
        if self.writer:
            self.writer.add_scalar("Loss/train", train_loss, epoch)
            self.writer.add_scalar("Accuracy/train", train_acc, epoch)
        else:
            pass

    def train(self, max_epochs: int):
        for epoch in range(self.epochs_run, max_epochs):
            self._run_epoch(epoch)
        # Close the writer
        self.writer.close()

In [12]:
set_seed()
Trainer = Trainer(model=model,
                  train_data=training_dataloader,
                  test_data=test_dataloader,
                  optimizer=optimizer,
                  loss_fn=loss_fn,
                  scheduler=scheduler,
                  writer=writer)

In [13]:
set_seed()
Trainer.train(100)

[GPUcuda] Epoch 0 | Batchsize: 12 | Steps: 46935
[GPUcuda] Epoch 1 | Batchsize: 12 | Steps: 46935
[GPUcuda] Epoch 2 | Batchsize: 12 | Steps: 46935


KeyboardInterrupt: 

In [ ]:
def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)

In [ ]:
save_model(model=model, target_dir="models/final_models", model_name="Dino_Run1.pt")

In [ ]:
model.eval()
output = model(ten_img)
print(output[0].argmax())
arg = int(output[0].argmax().cpu().detach().numpy())
print(arg)
print(spid_to_sp[cid_to_spid[arg]])

In [ ]:
top5_probabilities, top5_class_indices = torch.topk(output[0] * 100, k=5)
top5_probabilities = top5_probabilities.cpu().detach().numpy()
top5_class_indices = top5_class_indices.cpu().detach().numpy()

print(top5_probabilities)

for proba, cid in zip(top5_probabilities, top5_class_indices):
    species_id = cid_to_spid[cid]
    species = spid_to_sp[species_id]
    print(species_id, species, proba)